## **Import Library & Download Dataset**

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from google.colab import files
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 1. Setup Kaggle & Download
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

# Cek apakah folder dataset sudah ada
if not os.path.exists('cub_dataset'):
    print("Dataset belum ditemukan.")

    # Upload kaggle.json jika belum ada
    if not os.path.exists('kaggle.json'):
        print("Silakan Upload file kaggle.json Anda:")
        files.upload()
        !chmod 600 /content/kaggle.json

    print("\nMendownload dataset CUB-200-2011...")
    !kaggle datasets download -d wenewone/cub2002011 --force
    print("Mengekstrak dataset...")
    !unzip -q cub2002011.zip -d cub_dataset
    print("Selesai download & ekstrak.")

# 2. FIX PATH OTOMATIS (Agar tidak FileNotFoundError)
base_dir = 'cub_dataset'
possible_path_1 = os.path.join(base_dir, 'images')
possible_path_2 = os.path.join(base_dir, 'CUB_200_2011', 'images')

if os.path.exists(possible_path_1):
    data_dir = possible_path_1
elif os.path.exists(possible_path_2):
    data_dir = possible_path_2
else:
    # Fallback terakhir jika struktur zip berubah
    data_dir = 'cub_dataset/images'

print(f"✅ PATH DATASET DITEMUKAN DI: {data_dir}")

# Cek Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"✅ Menggunakan Device: {device}")

Dataset belum ditemukan.
Silakan Upload file kaggle.json Anda:


Saving kaggle.json to kaggle.json

Mendownload dataset CUB-200-2011...
Dataset URL: https://www.kaggle.com/datasets/wenewone/cub2002011
License(s): CC0-1.0
 97% 1.45G/1.49G [00:17<00:00, 43.9MB/s]
100% 1.49G/1.49G [00:17<00:00, 90.9MB/s]
Mengekstrak dataset...
Selesai download & ekstrak.
✅ PATH DATASET DITEMUKAN DI: cub_dataset/CUB_200_2011/images
✅ Menggunakan Device: cuda:0


## **Konfigurasi Hyperparameter & Augmentasi**

In [2]:
# --- KONFIGURASI SPEED & ACCURACY ---
BATCH_SIZE = 32     # Aman & Cepat
NUM_CLASSES = 200
IMG_SIZE = 320      # Resolusi Optimal (Detail cukup, ringan di memori)

# --- AUGMENTASI DATA ---
data_transforms = {
    'train': transforms.Compose([
        # Resize sedikit lebih besar dari target crop agar tidak ada border kosong
        transforms.Resize((350, 350)),
        # Random Crop ke 320px (memaksa model melihat bagian burung yang berbeda-beda)
        transforms.RandomCrop(IMG_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        # ColorJitter penting untuk dataset CUB agar model fokus ke bentuk, bukan warna background
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
print("✅ Konfigurasi Augmentasi Selesai")

✅ Konfigurasi Augmentasi Selesai


## **Split Dataset & Membuat DataLoader**

In [3]:
# Load Dataset Awal (Tanpa Transform Dulu)
full_dataset = datasets.ImageFolder(data_dir)

# Split 80% Train, 20% Val
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_data, val_data = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Wrapper Class (Agar Transform Train & Val tidak tertukar)
class TransformedDataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y
    def __len__(self):
        return len(self.subset)

# Terapkan Transform
train_dataset = TransformedDataset(train_data, transform=data_transforms['train'])
val_dataset = TransformedDataset(val_data, transform=data_transforms['val'])

# Buat DataLoader (pin_memory=True mempercepat transfer data ke GPU)
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True),
    'val': DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
}

dataset_sizes = {'train': train_size, 'val': val_size}

print(f"✅ Data Siap!")
print(f"   - Training: {train_size} gambar")
print(f"   - Validasi: {val_size} gambar")
print(f"   - Batch Size: {BATCH_SIZE}")
print(f"   - Image Size: {IMG_SIZE}x{IMG_SIZE}")

# Test ambil 1 batch untuk memastikan tidak ada error
images, labels = next(iter(dataloaders['train']))
print(f"   - Ukuran Tensor Batch: {images.shape}")
# Output harusnya: torch.Size([32, 3, 320, 320])

✅ Data Siap!
   - Training: 9436 gambar
   - Validasi: 2360 gambar
   - Batch Size: 32
   - Image Size: 320x320
   - Ukuran Tensor Batch: torch.Size([32, 3, 320, 320])


## **Training Loop dengan AMP (Automatic Mixed Precision)**

In [8]:
from torch.cuda.amp import autocast, GradScaler
import time # Import the time module
import copy # Import the copy module, as it's also used in this function
from tqdm import tqdm # Import tqdm for progress bar

def train_fast_amp(model, criterion, optimizer, scheduler=None, num_epochs=10, name="Model"):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    history = {'val_acc': []}

    # Inisialisasi Scaler untuk Mixed Precision
    scaler = GradScaler()

    print(f"\n{'='*10} TRAINING: {name} (AMP + Res 320px) {'='*10}")

    for epoch in range(num_epochs):
        model.train()
        train_corrects = 0

        # TQDM Progress Bar
        loop_train = tqdm(dataloaders['train'], leave=True, desc=f"Ep {epoch+1}/{num_epochs}")

        for inputs, labels in loop_train:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # --- MIXED PRECISION BLOCK (MAGIC HAPPENS HERE) ---
            with autocast():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

            # Backprop menggunakan Scaler
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            # --------------------------------------------------

            train_corrects += torch.sum(preds == labels.data)
            loop_train.set_postfix(loss=loss.item())

        # Step Scheduler di akhir epoch (jika bukan OneCycleLR)
        # Jika OneCycleLR, step harus di dalam batch loop.
        # Disini kita pakai step per epoch untuk simplifikasi fine tuning.
        if scheduler:
            scheduler.step()

        epoch_acc_train = train_corrects.double() / dataset_sizes['train']

        # Validasi
        model.eval()
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in dataloaders['val']:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        epoch_acc_val = val_corrects.double() / dataset_sizes['val']
        history['val_acc'].append(epoch_acc_val.item())

        print(f"Result: train_acc: {epoch_acc_train:.4f} - val_acc: {epoch_acc_val:.4f}")

        if epoch_acc_val > best_acc:
            best_acc = epoch_acc_val
            best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'Selesai dalam {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    model.load_state_dict(best_model_wts)
    return model, history

## **Eksekusi 3 Skenario**

### **TRAINING FROM SCRATCH**

In [9]:
NUM_EPOCHS = 10 # Define the number of epochs

# Pastikan fungsi train_fast_amp sudah dijalankan sebelumnya
criterion = nn.CrossEntropyLoss()

# --- SKENARIO 1: SCRATCH (Training dari Nol) ---
print("\n" + "="*40)
print(">>> MULAI SKENARIO 1: SCRATCH <<<")
print("="*40)
model_scratch = models.resnet50(weights=None) # Tanpa bobot
model_scratch.fc = nn.Linear(model_scratch.fc.in_features, NUM_CLASSES)
model_scratch = model_scratch.to(device)

# AdamW sangat bagus untuk start dari nol
optimizer_s = optim.AdamW(model_scratch.parameters(), lr=0.001)

model_scratch, hist_scratch = train_fast_amp(
    model_scratch, criterion, optimizer_s, num_epochs=NUM_EPOCHS, name="Scratch"
)


>>> MULAI SKENARIO 1: SCRATCH <<<


/tmp/ipython-input-239132057.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()



========== TRAINING: Scratch (AMP + Res 320px) ==========


Ep 1/10:   0%|          | 0/295 [00:00<?, ?it/s]/tmp/ipython-input-239132057.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Ep 1/10: 100%|██████████| 295/295 [01:59<00:00,  2.47it/s, loss=4.88]


Result: train_acc: 0.0099 - val_acc: 0.0127


...

Result: train_acc: 0.1017 - val_acc: 0.1153


Ep 9/10: 100%|██████████| 295/295 [01:48<00:00,  2.72it/s, loss=4.77]


Result: train_acc: 0.1325 - val_acc: 0.1131


Ep 10/10: 100%|██████████| 295/295 [01:48<00:00,  2.71it/s, loss=3.62]


Result: train_acc: 0.1424 - val_acc: 0.1428
Selesai dalam 20m 53s


### **TRANSFER LEARNING (FEATURE EXTRACTION)**

In [10]:
print("\n" + "="*40)
print(">>> MULAI SKENARIO 2: FEATURE EXTRACTION <<<")
print("="*40)
model_fe = models.resnet50(weights='IMAGENET1K_V1') # Pakai bobot ImageNet

# Freeze (Bekukan) semua layer kecuali yang terakhir
for param in model_fe.parameters():
    param.requires_grad = False

model_fe.fc = nn.Linear(model_fe.fc.in_features, NUM_CLASSES)
model_fe = model_fe.to(device)

# Optimizer hanya mengupdate layer FC (Classifier)
optimizer_fe = optim.AdamW(model_fe.fc.parameters(), lr=0.001)

model_fe, hist_fe = train_fast_amp(
    model_fe, criterion, optimizer_fe, num_epochs=NUM_EPOCHS, name="Feature Extraction"
)


>>> MULAI SKENARIO 2: FEATURE EXTRACTION <<<
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 195MB/s]
/tmp/ipython-input-239132057.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()



========== TRAINING: Feature Extraction (AMP + Res 320px) ==========


Ep 1/10:   0%|          | 0/295 [00:00<?, ?it/s]/tmp/ipython-input-239132057.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Ep 1/10: 100%|██████████| 295/295 [01:33<00:00,  3.15it/s, loss=3.92]


Result: train_acc: 0.1256 - val_acc: 0.2648


...

Result: train_acc: 0.6713 - val_acc: 0.5441


Ep 9/10: 100%|██████████| 295/295 [01:33<00:00,  3.15it/s, loss=1.7]


Result: train_acc: 0.6878 - val_acc: 0.5229


Ep 10/10: 100%|██████████| 295/295 [01:34<00:00,  3.11it/s, loss=1.37]


Result: train_acc: 0.7041 - val_acc: 0.5297
Selesai dalam 18m 20s


### **FINE TUNING**

In [11]:
print("\n" + "="*40)
print(">>> MULAI SKENARIO 3: FINE TUNING <<<")
print("="*40)
model_ft = models.resnet50(weights='IMAGENET1K_V1')
model_ft.fc = nn.Linear(model_ft.fc.in_features, NUM_CLASSES)
model_ft = model_ft.to(device)

# Settingan "Pro": Learning Rate berbeda tiap layer
# Layer awal (fitur dasar) berubah pelan, Layer akhir (keputusan) berubah cepat
optimizer_ft = optim.AdamW([
    {'params': model_ft.conv1.parameters(), 'lr': 1e-5},
    {'params': model_ft.layer1.parameters(), 'lr': 1e-5},
    {'params': model_ft.layer2.parameters(), 'lr': 5e-5},
    {'params': model_ft.layer3.parameters(), 'lr': 1e-4},
    {'params': model_ft.layer4.parameters(), 'lr': 2e-4},
    {'params': model_ft.fc.parameters(), 'lr': 1e-3}
], weight_decay=0.01)

# Scheduler: Menurunkan LR setiap 3 epoch agar model makin teliti
scheduler_ft = optim.lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.2)

model_ft, hist_ft = train_fast_amp(
    model_ft, criterion, optimizer_ft, scheduler=scheduler_ft, num_epochs=NUM_EPOCHS, name="Fine Tuning"
)


>>> MULAI SKENARIO 3: FINE TUNING <<<


/tmp/ipython-input-239132057.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()



========== TRAINING: Fine Tuning (AMP + Res 320px) ==========


Ep 1/10:   0%|          | 0/295 [00:00<?, ?it/s]/tmp/ipython-input-239132057.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Ep 1/10: 100%|██████████| 295/295 [01:48<00:00,  2.72it/s, loss=1.48]


Result: train_acc: 0.3488 - val_acc: 0.5195


...

Result: train_acc: 0.9625 - val_acc: 0.8318


Ep 9/10: 100%|██████████| 295/295 [01:47<00:00,  2.73it/s, loss=0.197]


Result: train_acc: 0.9706 - val_acc: 0.8318


Ep 10/10: 100%|██████████| 295/295 [01:48<00:00,  2.73it/s, loss=0.283]


Result: train_acc: 0.9721 - val_acc: 0.8314
Selesai dalam 20m 43s


## **Visualisasi Perbandingan**

In [12]:
acc_scratch_max = max(hist_scratch['val_acc']) * 100
acc_fe_max = max(hist_fe['val_acc']) * 100
acc_ft_max = max(hist_ft['val_acc']) * 100

print("\n=== Ringkasan Akurasi Tertinggi (Optimized Speed) ===")
print(f"Scratch: {acc_scratch_max:.2f}%")
print(f"Feature Extraction: {acc_fe_max:.2f}%")
print(f"Fine Tuning: {acc_ft_max:.2f}%")


=== Ringkasan Akurasi Tertinggi (Optimized Speed) ===
Scratch: 14.28%
Feature Extraction: 54.41%
Fine Tuning: 83.22%
